In [2]:
pip install catboost

Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for catboost from https://files.pythonhosted.org/packages/35/7e/35fa1a7cf6925ff438e849cca50c88b8d28e02d9c3486442f2f85b86184a/catboost-1.2.5-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for graphviz from https://files.pythonhosted.org/packages/00/be/d59db2d1d52697c6adc9eacaf50e8965b6345cc143f671e1ed068818d5cf/graphviz-0.20.3-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/101.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/101.1 MB ? eta -:--:--
   ---------------------------------------- 0.2/101.1 MB 3.1 MB/s eta 0:00:34
   ---------------------------------------- 0.7/101.1 MB 6.6 MB/s eta 0:00:16
    --------------------------------------- 1.3/101.1 MB 8.2 MB/s eta 0:00:13
    --------------------------------------- 1.7/101.1 MB 8.9 MB/s eta 0:00:12
    --------------------------------------- 2.1/101.1 MB

In [4]:
pip install lazypredict

Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for lazypredict from https://files.pythonhosted.org/packages/bb/df/936639581c018ac59b1096d34e42d39e967d97bf09b1fdb1645e64852e36/lazypredict-0.2.12-py2.py3-none-any.whl.metadata
  Obtaining dependency information for lightgbm from https://files.pythonhosted.org/packages/ca/b4/57f3f253721e0a16ea28c49acca92c5b1198eb94fbbb8328d6dabc61d2e0/lightgbm-4.4.0-py3-none-win_amd64.whl.metadata
  Obtaining dependency information for xgboost from https://files.pythonhosted.org/packages/a2/48/d5da8591891327b0faf08179d420fba3893c6134bdd449497c5329e4cb01/xgboost-2.1.0-py3-none-win_amd64.whl.metadata
   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
   ------- -------------------------------- 0.3/1.4 MB 8.3 MB/s eta 0:00:01
   -------------------------- ------------- 1.0/1.4 MB 12.3 MB/s eta 0:00:01
   ---------------------------------------  1.4/1.4 MB 10.2 MB/s eta 0:00:01

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [1]:
import pandas as pd
import numpy as np
import ast
import catboost
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, balanced_accuracy_score, precision_score, f1_score
from sklearn.model_selection import train_test_split
from lazypredict.Supervised import LazyClassifier
import pickle
import xgboost as xgb
import lightgbm as lgb
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import OneHotEncoder

In [2]:
data = pd.read_csv('game_data_final_win_loss.csv')
data.head(5)

,m1,m2,m3,m4,m5,m6,m7,m8,m9,m10,...,side to play,fens,plies,w_elo,b_elo,w_time,b_time,inc_time,stock_evals,result
0,0,0,0,0,0,0,0,0,0,0,...,0,rn3r1k/pp3ppp/4p3/3p1N2/8/2NP2PP/PPP3P1/2KR3R ...,34,2059,1941,833.51,559.17,10.00,"{'e6f5': 239, 'h8g8': 458, 'b8c6': 476, 'h7h6'...",1
1,0,0,0,0,0,0,0,0,0,0,...,0,3r3k/1p3ppp/p1n1r3/3N1p2/1PP5/P2P2PP/3K2P1/4R2...,46,2059,1941,781.44,483.19,10.00,"{'g7g5': 232, 'h8g8': 253, 'b7b5': 254, 'g7g6'...",1
2,1,1,2,0,0,0,1,1,0,0,...,1,4k3/1p6/p5pp/3p1p2/1PP4P/P1K3P1/6P1/8 w - - 0 34,67,2059,1941,702.54,307.39,10.00,"{'c4d5': 206, 'c3d4': 131, 'c3d3': 130, 'a3a4'...",1
3,1,1,0,0,0,0,1,1,0,0,...,1,8/1p6/p2k2pp/3P1p2/PP5P/2K3P1/6P1/8 w - - 1 36,71,2059,1941,684.55,316.30,10.00,"{'c3c4': 284, 'h4h5': 283, 'c3d4': 229, 'g3g4'...",1
4,0,0,0,0,0,0,0,0,0,0,...,0,r2q1rk1/1pp1b1pp/2np4/p7/2Pp4/P4BPP/1P1BQP2/4R...,36,2743,2319,602.32,790.94,0.00,"{'e7f6': 34, 'f8e8': 72, 'e7g5': 82, 'f8f3': 1...",1


In [3]:
idx = 0
samp_eval = ast.literal_eval(data['stock_evals'].values[idx])
print(samp_eval)
move_evals = np.array(list(samp_eval.values()))
loss = np.abs(move_evals - move_evals[0])
is_white = data['plies'][idx] % 2
alpha = (is_white*data['w_elo'][idx] + (1 - is_white)*data['b_elo'][idx])/3300
print(alpha)
expo_loss = np.exp(-4*alpha*0.01*loss)
print(loss)
print(expo_loss/sum(expo_loss))
#print(len(expo_loss))
total_expo_loss = np.sum(expo_loss)
expected_loss = np.dot(expo_loss, loss)/total_expo_loss
print(total_expo_loss)
print(len(loss))

{'e6f5': 239, 'h8g8': 458, 'b8c6': 476, 'h7h6': 477, 'b8a6': 481, 'b7b5': 481, 'f7f6': 489, 'a7a5': 496, 'f8d8': 501, 'f8g8': 502, 'f8e8': 502, 'g7g6': 503, 'a7a6': 506, 'f8c8': 509, 'b7b6': 509, 'b8d7': 512, 'h7h5': 525, 'g7g5': 573, 'e6e5': 624, 'd5d4': 614}
0.5881818181818181
[  0 219 237 238 242 242 250 257 262 263 263 264 267 270 270 273 286 334
 385 375]
[9.59500938e-01 5.55079170e-03 3.63442043e-03 3.54991047e-03
 3.23106981e-03 3.23106981e-03 2.67672789e-03 2.27028234e-03
 2.01832476e-03 1.97139333e-03 1.97139333e-03 1.92555318e-03
 1.79432961e-03 1.67204873e-03 1.67204873e-03 1.55810110e-03
 1.14753171e-03 3.70947106e-04 1.11739310e-04 1.41378586e-04]
1.0422084651584114
20


In [4]:
stock_evals = data['stock_evals'].values
w_elos = data['w_elo'].values
b_elos = data['b_elo'].values
plies = data['plies'].values
w_plus_all = []
b_plus_all = []
equals_all = []
pos_evals = []
expected_losses = []
std_losses = []
thres = 20
f = 4
for i, eval in enumerate(stock_evals):
    dict = ast.literal_eval(eval)
    move_evals = np.array(list(dict.values()))
    pos_evals.append(move_evals[0])
    loss = np.abs(move_evals - move_evals[0])
    is_white = plies[i] % 2
    alpha = (is_white*w_elos[i] + (1 - is_white)*b_elos[i])/3000
    expo_loss = np.exp(-f*alpha*0.01*loss)
    total_expo_loss = np.sum(expo_loss)
    expected_loss = np.dot(expo_loss, loss)/total_expo_loss
    expected_losses.append(expected_loss)
    std_losses.append(np.sqrt(np.dot(expo_loss, loss*loss)/total_expo_loss- expected_loss*expected_loss))
    w_plus = b_plus = equals = 0
    for item in dict.values():
        if item > thres:
            w_plus += 1
        elif item < -thres:
            b_plus += 1
        else:
            equals += 1
    w_plus_all.append(w_plus)
    b_plus_all.append(b_plus)
    equals_all.append(equals)

In [5]:
data_x = data.iloc[:, 1:-2]
data_x['expected_losses'] = expected_losses
data_x['std_losses'] = std_losses
data_x['pos_evals'] = pos_evals
data_x['white_plus'] = w_plus_all
data_x['black_plus'] = b_plus_all
data_x['equals'] = equals_all
data_x.head(5)

,m2,m3,m4,m5,m6,m7,m8,m9,m10,m11,...,b_elo,w_time,b_time,inc_time,expected_losses,std_losses,pos_evals,white_plus,black_plus,equals
0,0,0,0,0,0,0,0,0,0,0,...,1941,833.51,559.17,10.00,5.72,37.51,239,20,0,0
1,0,0,0,0,0,0,0,0,0,0,...,1941,781.44,483.19,10.00,42.47,34.43,232,37,0,0
2,1,2,0,0,0,1,1,0,0,0,...,1941,702.54,307.39,10.00,30.10,43.83,206,8,3,1
3,1,0,0,0,0,1,1,0,0,0,...,1941,684.55,316.30,10.00,6.09,17.35,284,3,7,1
4,0,0,0,0,0,0,0,0,0,0,...,2319,602.32,790.94,0.00,17.85,28.57,34,32,0,0


In [6]:
x = data_x.values
y = data['result'].values + 1
# x_train, x_test_val, y_train, y_test_val = train_test_split(x, y, test_size = 0.3, shuffle = True, random_state = 0)
# x_val, x_test, y_val, y_test = train_test_split(x_test_val, y_test_val, test_size = 1.0)
#x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, shuffle = True, random_state = 0)

In [7]:
x[0]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 0, 1, 2, 1, 2,
       2, 1, 1, 0, 0, 1, 1, 0, 0, 0,
       'rn3r1k/pp3ppp/4p3/3p1N2/8/2NP2PP/PPP3P1/2KR3R b - - 0 17', 34,
       2059, 1941, 833.512, 559.172, 10.0, 5.7185991109676815,
       37.509976506943715, 239, 20, 0, 0], dtype=object)

In [50]:
x[0].shape

(43,)

In [8]:
x = np.array([[float(xij) for xij in xi[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43]]] for xi in x])

In [28]:
x[0]

array([0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 2.00000000e+00,
       2.00000000e+00, 0.00000000e+00, 1.00000000e+00, 2.00000000e+00,
       1.00000000e+00, 2.00000000e+00, 2.00000000e+00, 1.00000000e+00,
       1.00000000e+00, 0.00000000e+00, 0.00000000e+00, 1.00000000e+00,
       1.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       3.40000000e+01, 2.05900000e+03, 1.94100000e+03, 8.33512000e+02,
       5.59172000e+02, 1.00000000e+01, 5.71859911e+00, 3.75099765e+01,
       2.39000000e+02, 2.00000000e+01, 0.00000000e+00])

In [9]:
x_train, x_test_val, y_train, y_test_val = train_test_split(x, y, test_size = 0.3, shuffle = True, random_state = 0)
x_val, x_test, y_val, y_test = train_test_split(x_test_val, y_test_val, test_size = 0.3)

BASIC STOCKFISH BASED MODEL

In [10]:
bd = 50 #equality threshold
test_evals = x_test[:, 42]
y_pred_stock = np.array([it > bd for it in test_evals]).astype(int) - np.array([it < -bd for it in test_evals]).astype(int) + 1
#y_true = ohe.inverse_transform(y_test).reshape(-1)
print(f'Accuracy = {accuracy_score(y_test, y_pred_stock)*100}%')
print(f'Balanced Accuracy = {balanced_accuracy_score(y_test, y_pred_stock)*100}%')
#print(f'Precision = {precision_score(y_test, y_pred_stock, average = mode)*100}%')
#print(f'F1 Score = {f1_score(y_test, y_pred_stock, average = mode)*100}%')
confusion_matrix(y_test, y_pred_stock)

Accuracy = 16.327436650502015%
Balanced Accuracy = 33.35410811936995%


array([[    0, 11512,    78],
       [    0,  4743,    11],
       [    0, 12900,    38]], dtype=int64)

In [11]:
ohe = OneHotEncoder(sparse_output = False)
y_ohe = ohe.fit_transform(y.reshape(-1, 1))
print(ohe.categories_)

[array([0, 1, 2], dtype=int64)]


In [12]:
del model_nn

NameError: name 'model_nn' is not defined

In [13]:
x_train_nn, x_test_val_nn, y_train_nn, y_test_val_nn = train_test_split(x, y_ohe, test_size = 0.3, shuffle = True, random_state = 0)
x_val_nn, x_test_nn, y_val_nn, y_test_nn = train_test_split(x_test_val_nn, y_test_val_nn, shuffle = False, test_size = 0.6)

In [14]:
x_val_nn[0]

array([0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 1.00000000e+00,
       1.00000000e+00, 1.00000000e+00, 0.00000000e+00, 1.00000000e+00,
       1.00000000e+00, 1.00000000e+00, 1.00000000e+00, 1.00000000e+00,
       1.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       3.80000000e+01, 2.70500000e+03, 2.08700000e+03, 5.72711000e+02,
       6.89103000e+02, 0.00000000e+00, 2.83095580e+01, 2.84760874e+01,
       1.98000000e+02, 3.60000000e+01, 0.00000000e+00])

In [15]:
ni = len(x_train[0])
no = len(y_ohe[0])
model_nn = Sequential([
    Dense(256, input_shape = (ni,), activation = 'relu'),
    #Dropout(0.1),
    Dense(256, activation = 'relu'),
    #Dropout(0.1),
    #Dense(400, activation = 'relu'),
    Dense(256, activation = 'relu'),
    Dense(no, activation = 'softmax')
])

In [16]:
model_nn.compile(
    optimizer = 'adam',
    loss = 'categorical_crossentropy',
    metrics = ['accuracy']
)

In [17]:
early_stopping_callback = EarlyStopping(monitor = 'val_loss', patience = 5)
model_nn.fit(x_train_nn, y_train_nn, epochs = 50, validation_data = (x_val_nn, y_val_nn), batch_size = 32, callbacks = [early_stopping_callback])

Epoch 1/50


7117/7117 [==============================] - 43s 6ms/step - loss: 1.2338 - accuracy: 0.7301 - val_loss: 0.6056 - val_accuracy: 0.7486
Epoch 2/50
7117/7117 [==============================] - 38s 5ms/step - loss: 0.6122 - accuracy: 0.7520 - val_loss: 0.5634 - val_accuracy: 0.7766
Epoch 3/50
7117/7117 [==============================] - 41s 6ms/step - loss: 0.5718 - accuracy: 0.7704 - val_loss: 0.5857 - val_accuracy: 0.7579
Epoch 4/50
7117/7117 [==============================] - 41s 6ms/step - loss: 0.5599 - accuracy: 0.7762 - val_loss: 0.5533 - val_accuracy: 0.7836
Epoch 5/50
7117/7117 [==============================] - 38s 5ms/step - loss: 0.5549 - accuracy: 0.7793 - val_loss: 0.5457 - val_accuracy: 0.7813
Epoch 6/50
7117/7117 [==============================] - 39s 5ms/step - loss: 0.5483 - accuracy: 0.7808 - val_loss: 0.5557 - val_accuracy: 0.7754
Epoch 7/50
7117/7117 [==============================] - 38s 5ms/step - loss: 0.5456 - accuracy: 0.7817 - val_loss: 0.5429 - val_

In [18]:
mode = 'weighted'
y_pred_nn = model_nn.predict(x_test_nn)
y_pred_nn = np.argmax(y_pred_nn, axis = 1)
y_true_nn = ohe.inverse_transform(y_test_nn).reshape(-1)
print(f'Accuracy = {accuracy_score(y_true_nn, y_pred_nn)*100}%')
print(f'Balanced Accuracy = {balanced_accuracy_score(y_true_nn, y_pred_nn)*100}%')
#print(f'Precision = {precision_score(y_true_nn, y_pred_nn, average = mode)*100}%')
#print(f'F1 Score = {f1_score(y_true_nn, y_pred_nn, average = mode)*100}%')
confusion_matrix(y_true_nn, y_pred_nn)

1831/1831 [==============================] - 6s 3ms/step
Accuracy = 78.75962638526032%
Balanced Accuracy = 74.2440660531357%


array([[19390,  1881,  2291],
       [ 1961,  5356,  2099],
       [ 2093,  2114, 21378]], dtype=int64)

In [19]:
#model_nn.save('model_nn.keras')
model_nn.save('model_nn.h5')

In [20]:
from sklearn.preprocessing import StandardScaler

# Initialize the scaler
scaler = StandardScaler()

# Fit the scaler on the training data and transform it
x_train_scaled = scaler.fit_transform(x_train)

# Use the same scaler to transform the validation and test data
x_val_scaled = scaler.transform(x_val)
x_test_scaled = scaler.transform(x_test)

In [21]:
model_cat = CatBoostClassifier(iterations = 5000, learning_rate = 0.05, depth = 6, eval_metric = 'Accuracy', random_seed = 42, logging_level = 'Verbose', use_best_model = True, early_stopping_rounds = 300)
model_cat.fit(x_train_scaled, y_train, eval_set = (x_val_scaled, y_val))
y_pred_cat = model_cat.predict(x_test_scaled)
print(f'The accuracy of the catboost model is: {accuracy_score(y_test, y_pred_cat)}')

0:	learn: 0.7621573	test: 0.7616762	best: 0.7616762 (0)	total: 314ms	remaining: 26m 12s
1:	learn: 0.7640981	test: 0.7629934	best: 0.7629934 (1)	total: 457ms	remaining: 19m 3s
2:	learn: 0.7661970	test: 0.7647352	best: 0.7647352 (2)	total: 600ms	remaining: 16m 40s
3:	learn: 0.7705528	test: 0.7699896	best: 0.7699896 (3)	total: 736ms	remaining: 15m 19s
4:	learn: 0.7719579	test: 0.7712191	best: 0.7712191 (4)	total: 873ms	remaining: 14m 31s
5:	learn: 0.7731522	test: 0.7729754	best: 0.7729754 (5)	total: 1.02s	remaining: 14m 12s
6:	learn: 0.7752554	test: 0.7740292	best: 0.7740292 (6)	total: 1.16s	remaining: 13m 46s
7:	learn: 0.7757253	test: 0.7742927	best: 0.7742927 (7)	total: 1.29s	remaining: 13m 24s
8:	learn: 0.7759316	test: 0.7738097	best: 0.7742927 (7)	total: 1.42s	remaining: 13m 9s
9:	learn: 0.7757648	test: 0.7737512	best: 0.7742927 (7)	total: 1.55s	remaining: 12m 53s
10:	learn: 0.7764234	test: 0.7743952	best: 0.7743952 (10)	total: 1.69s	remaining: 12m 46s
11:	learn: 0.7773631	test: 0.775

92:	learn: 0.7925864	test: 0.7902024	best: 0.7902024 (92)	total: 12.4s	remaining: 10m 55s
93:	learn: 0.7926391	test: 0.7900268	best: 0.7902024 (92)	total: 12.6s	remaining: 10m 55s
94:	learn: 0.7926566	test: 0.7902024	best: 0.7902024 (92)	total: 12.7s	remaining: 10m 55s
95:	learn: 0.7925996	test: 0.7901878	best: 0.7902024 (92)	total: 12.8s	remaining: 10m 55s
96:	learn: 0.7926566	test: 0.7901292	best: 0.7902024 (92)	total: 13s	remaining: 10m 55s
97:	learn: 0.7926830	test: 0.7902317	best: 0.7902317 (97)	total: 13.1s	remaining: 10m 55s
98:	learn: 0.7928103	test: 0.7905098	best: 0.7905098 (98)	total: 13.2s	remaining: 10m 55s
99:	learn: 0.7928323	test: 0.7905683	best: 0.7905683 (99)	total: 13.4s	remaining: 10m 55s
100:	learn: 0.7927884	test: 0.7907001	best: 0.7907001 (100)	total: 13.5s	remaining: 10m 54s
101:	learn: 0.7930035	test: 0.7907001	best: 0.7907001 (100)	total: 13.6s	remaining: 10m 54s
102:	learn: 0.7930255	test: 0.7907586	best: 0.7907586 (102)	total: 13.8s	remaining: 10m 54s
103:	l

182:	learn: 0.7966304	test: 0.7941981	best: 0.7941981 (182)	total: 23.2s	remaining: 10m 11s
183:	learn: 0.7966787	test: 0.7941250	best: 0.7941981 (182)	total: 23.3s	remaining: 10m 11s
184:	learn: 0.7966260	test: 0.7940811	best: 0.7941981 (182)	total: 23.5s	remaining: 10m 10s
185:	learn: 0.7966348	test: 0.7941103	best: 0.7941981 (182)	total: 23.6s	remaining: 10m 10s
186:	learn: 0.7966304	test: 0.7941835	best: 0.7941981 (182)	total: 23.7s	remaining: 10m 9s
187:	learn: 0.7967534	test: 0.7941103	best: 0.7941981 (182)	total: 23.8s	remaining: 10m 9s
188:	learn: 0.7967490	test: 0.7940371	best: 0.7941981 (182)	total: 23.9s	remaining: 10m 9s
189:	learn: 0.7967929	test: 0.7940957	best: 0.7941981 (182)	total: 24.1s	remaining: 10m 9s
190:	learn: 0.7968895	test: 0.7941103	best: 0.7941981 (182)	total: 24.2s	remaining: 10m 8s
191:	learn: 0.7969597	test: 0.7941689	best: 0.7941981 (182)	total: 24.3s	remaining: 10m 8s
192:	learn: 0.7969070	test: 0.7942860	best: 0.7942860 (192)	total: 24.4s	remaining: 10

274:	learn: 0.7987864	test: 0.7959252	best: 0.7959252 (274)	total: 34s	remaining: 9m 44s
275:	learn: 0.7987644	test: 0.7958813	best: 0.7959252 (274)	total: 34.1s	remaining: 9m 44s
276:	learn: 0.7987951	test: 0.7958960	best: 0.7959252 (274)	total: 34.3s	remaining: 9m 44s
277:	learn: 0.7987864	test: 0.7959252	best: 0.7959252 (274)	total: 34.4s	remaining: 9m 43s
278:	learn: 0.7988039	test: 0.7957642	best: 0.7959252 (274)	total: 34.5s	remaining: 9m 43s
279:	learn: 0.7988522	test: 0.7957789	best: 0.7959252 (274)	total: 34.6s	remaining: 9m 43s
280:	learn: 0.7988830	test: 0.7957350	best: 0.7959252 (274)	total: 34.7s	remaining: 9m 43s
281:	learn: 0.7989313	test: 0.7958228	best: 0.7959252 (274)	total: 34.8s	remaining: 9m 42s
282:	learn: 0.7988742	test: 0.7958960	best: 0.7959252 (274)	total: 35s	remaining: 9m 42s
283:	learn: 0.7989181	test: 0.7958960	best: 0.7959252 (274)	total: 35.1s	remaining: 9m 42s
284:	learn: 0.7989971	test: 0.7959545	best: 0.7959545 (284)	total: 35.2s	remaining: 9m 42s
285

366:	learn: 0.8004769	test: 0.7968620	best: 0.7969351 (361)	total: 44.6s	remaining: 9m 23s
367:	learn: 0.8005032	test: 0.7969644	best: 0.7969644 (367)	total: 44.7s	remaining: 9m 23s
368:	learn: 0.8005339	test: 0.7970522	best: 0.7970522 (368)	total: 44.9s	remaining: 9m 23s
369:	learn: 0.8005339	test: 0.7969644	best: 0.7970522 (368)	total: 45s	remaining: 9m 22s
370:	learn: 0.8005822	test: 0.7970083	best: 0.7970522 (368)	total: 45.1s	remaining: 9m 22s
371:	learn: 0.8006130	test: 0.7970230	best: 0.7970522 (368)	total: 45.2s	remaining: 9m 22s
372:	learn: 0.8006393	test: 0.7971693	best: 0.7971693 (372)	total: 45.3s	remaining: 9m 22s
373:	learn: 0.8006437	test: 0.7971401	best: 0.7971693 (372)	total: 45.4s	remaining: 9m 22s
374:	learn: 0.8006086	test: 0.7969937	best: 0.7971693 (372)	total: 45.6s	remaining: 9m 21s
375:	learn: 0.8006788	test: 0.7969498	best: 0.7971693 (372)	total: 45.7s	remaining: 9m 22s
376:	learn: 0.8006569	test: 0.7970376	best: 0.7971693 (372)	total: 45.8s	remaining: 9m 21s
3

...

2968:	learn: 0.8210790	test: 0.8020725	best: 0.8022189 (2906)	total: 4m 46s	remaining: 3m 16s
2969:	learn: 0.8211054	test: 0.8020579	best: 0.8022189 (2906)	total: 4m 47s	remaining: 3m 16s
2970:	learn: 0.8211273	test: 0.8020725	best: 0.8022189 (2906)	total: 4m 47s	remaining: 3m 16s
2971:	learn: 0.8211449	test: 0.8020725	best: 0.8022189 (2906)	total: 4m 47s	remaining: 3m 16s
2972:	learn: 0.8211712	test: 0.8021164	best: 0.8022189 (2906)	total: 4m 47s	remaining: 3m 15s
2973:	learn: 0.8211449	test: 0.8020871	best: 0.8022189 (2906)	total: 4m 47s	remaining: 3m 15s
2974:	learn: 0.8211405	test: 0.8020871	best: 0.8022189 (2906)	total: 4m 47s	remaining: 3m 15s
2975:	learn: 0.8211581	test: 0.8021018	best: 0.8022189 (2906)	total: 4m 47s	remaining: 3m 15s
2976:	learn: 0.8211756	test: 0.8021603	best: 0.8022189 (2906)	total: 4m 47s	remaining: 3m 15s
2977:	learn: 0.8211932	test: 0.8021896	best: 0.8022189 (2906)	total: 4m 47s	remaining: 3m 15s
2978:	learn: 0.8212415	test: 0.8022042	best: 0.8022189 (2906

3057:	learn: 0.8217201	test: 0.8023506	best: 0.8023652 (3056)	total: 4m 55s	remaining: 3m 7s
3058:	learn: 0.8217333	test: 0.8023652	best: 0.8023652 (3056)	total: 4m 55s	remaining: 3m 7s
3059:	learn: 0.8217421	test: 0.8024384	best: 0.8024384 (3059)	total: 4m 56s	remaining: 3m 7s
3060:	learn: 0.8217421	test: 0.8023067	best: 0.8024384 (3059)	total: 4m 56s	remaining: 3m 7s
3061:	learn: 0.8217728	test: 0.8023799	best: 0.8024384 (3059)	total: 4m 56s	remaining: 3m 7s
3062:	learn: 0.8217772	test: 0.8023652	best: 0.8024384 (3059)	total: 4m 56s	remaining: 3m 7s
3063:	learn: 0.8217684	test: 0.8023506	best: 0.8024384 (3059)	total: 4m 56s	remaining: 3m 7s
3064:	learn: 0.8217508	test: 0.8023360	best: 0.8024384 (3059)	total: 4m 56s	remaining: 3m 7s
3065:	learn: 0.8218167	test: 0.8023360	best: 0.8024384 (3059)	total: 4m 56s	remaining: 3m 7s
3066:	learn: 0.8218123	test: 0.8022774	best: 0.8024384 (3059)	total: 4m 56s	remaining: 3m 7s
3067:	learn: 0.8218430	test: 0.8023067	best: 0.8024384 (3059)	total: 4

3147:	learn: 0.8223085	test: 0.8023799	best: 0.8024384 (3059)	total: 5m 4s	remaining: 2m 59s
3148:	learn: 0.8223348	test: 0.8023945	best: 0.8024384 (3059)	total: 5m 4s	remaining: 2m 59s
3149:	learn: 0.8223173	test: 0.8023652	best: 0.8024384 (3059)	total: 5m 5s	remaining: 2m 59s
3150:	learn: 0.8223304	test: 0.8023360	best: 0.8024384 (3059)	total: 5m 5s	remaining: 2m 59s
3151:	learn: 0.8223348	test: 0.8023506	best: 0.8024384 (3059)	total: 5m 5s	remaining: 2m 58s
3152:	learn: 0.8223304	test: 0.8023652	best: 0.8024384 (3059)	total: 5m 5s	remaining: 2m 58s
3153:	learn: 0.8223260	test: 0.8023067	best: 0.8024384 (3059)	total: 5m 5s	remaining: 2m 58s
3154:	learn: 0.8223392	test: 0.8022481	best: 0.8024384 (3059)	total: 5m 5s	remaining: 2m 58s
3155:	learn: 0.8223436	test: 0.8023067	best: 0.8024384 (3059)	total: 5m 5s	remaining: 2m 58s
3156:	learn: 0.8223524	test: 0.8023213	best: 0.8024384 (3059)	total: 5m 5s	remaining: 2m 58s
3157:	learn: 0.8223963	test: 0.8023213	best: 0.8024384 (3059)	total: 5

3237:	learn: 0.8229408	test: 0.8022774	best: 0.8024384 (3059)	total: 5m 14s	remaining: 2m 50s
3238:	learn: 0.8229671	test: 0.8022774	best: 0.8024384 (3059)	total: 5m 14s	remaining: 2m 50s
3239:	learn: 0.8229452	test: 0.8022774	best: 0.8024384 (3059)	total: 5m 14s	remaining: 2m 50s
3240:	learn: 0.8229539	test: 0.8022481	best: 0.8024384 (3059)	total: 5m 14s	remaining: 2m 50s
3241:	learn: 0.8229583	test: 0.8022335	best: 0.8024384 (3059)	total: 5m 14s	remaining: 2m 50s
3242:	learn: 0.8229847	test: 0.8022335	best: 0.8024384 (3059)	total: 5m 14s	remaining: 2m 50s
3243:	learn: 0.8229803	test: 0.8022481	best: 0.8024384 (3059)	total: 5m 14s	remaining: 2m 50s
3244:	learn: 0.8229715	test: 0.8022189	best: 0.8024384 (3059)	total: 5m 14s	remaining: 2m 50s
3245:	learn: 0.8229715	test: 0.8022774	best: 0.8024384 (3059)	total: 5m 15s	remaining: 2m 50s
3246:	learn: 0.8229671	test: 0.8022189	best: 0.8024384 (3059)	total: 5m 15s	remaining: 2m 50s
3247:	learn: 0.8229496	test: 0.8022189	best: 0.8024384 (3059

3325:	learn: 0.8234326	test: 0.8022628	best: 0.8024384 (3059)	total: 5m 22s	remaining: 2m 42s
3326:	learn: 0.8233974	test: 0.8022774	best: 0.8024384 (3059)	total: 5m 22s	remaining: 2m 42s
3327:	learn: 0.8234018	test: 0.8022481	best: 0.8024384 (3059)	total: 5m 22s	remaining: 2m 42s
3328:	learn: 0.8233930	test: 0.8022189	best: 0.8024384 (3059)	total: 5m 22s	remaining: 2m 42s
3329:	learn: 0.8234062	test: 0.8022481	best: 0.8024384 (3059)	total: 5m 23s	remaining: 2m 42s
3330:	learn: 0.8233930	test: 0.8022042	best: 0.8024384 (3059)	total: 5m 23s	remaining: 2m 41s
3331:	learn: 0.8234150	test: 0.8022042	best: 0.8024384 (3059)	total: 5m 23s	remaining: 2m 41s
3332:	learn: 0.8234238	test: 0.8022335	best: 0.8024384 (3059)	total: 5m 23s	remaining: 2m 41s
3333:	learn: 0.8234765	test: 0.8022189	best: 0.8024384 (3059)	total: 5m 23s	remaining: 2m 41s
3334:	learn: 0.8234765	test: 0.8021896	best: 0.8024384 (3059)	total: 5m 23s	remaining: 2m 41s
3335:	learn: 0.8234940	test: 0.8021750	best: 0.8024384 (3059

In [22]:
clf = LazyClassifier(verbose = 1, ignore_warnings = False, custom_metric = None)
models, predictions = clf.fit(x_train, x_test, y_train, y_test)
print(predictions)

  3%|██▊                                                                                | 1/29 [00:43<20:22, 43.66s/it]

ROC AUC couldn't be calculated for AdaBoostClassifier
multi_class must be in ('ovo', 'ovr')
{'Model': 'AdaBoostClassifier', 'Accuracy': 0.7775766682603648, 'Balanced Accuracy': 0.7223864913171788, 'ROC AUC': None, 'F1 Score': 0.7740504017957397, 'Time taken': 43.65526628494263}


  7%|█████▋                                                                             | 2/29 [01:59<28:10, 62.62s/it]

ROC AUC couldn't be calculated for BaggingClassifier
multi_class must be in ('ovo', 'ovr')
{'Model': 'BaggingClassifier', 'Accuracy': 0.7896318557475582, 'Balanced Accuracy': 0.7397540103365415, 'ROC AUC': None, 'F1 Score': 0.7871439186328666, 'Time taken': 75.88879895210266}


 10%|████████▌                                                                          | 3/29 [02:00<14:55, 34.43s/it]

...

ROC AUC couldn't be calculated for SVC
multi_class must be in ('ovo', 'ovr')
{'Model': 'SVC', 'Accuracy': 0.7859435830885868, 'Balanced Accuracy': 0.7254141219868994, 'ROC AUC': None, 'F1 Score': 0.7808736779124322, 'Time taken': 4278.395513296127}
StackingClassifier model failed to execute
StackingClassifier.__init__() missing 1 required positional argument: 'estimators'


 97%|████████████████████████████████████████████████████████████████████████████▎  | 28/29 [3:34:21<12:52, 772.50s/it]

ROC AUC couldn't be calculated for XGBClassifier
multi_class must be in ('ovo', 'ovr')
{'Model': 'XGBClassifier', 'Accuracy': 0.8020968513079708, 'Balanced Accuracy': 0.7516757454619851, 'ROC AUC': None, 'F1 Score': 0.7992811488012096, 'Time taken': 5.3035728931427}
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.029585 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2373
[LightGBM] [Info] Number of data points in the train set: 227743, number of used features: 39
[LightGBM] [Info] Start training from score -0.911343
[LightGBM] [Info] Start training from score -1.830111
[LightGBM] [Info] Start training from score -0.826403


100%|███████████████████████████████████████████████████████████████████████████████| 29/29 [3:34:27<00:00, 443.70s/it]

ROC AUC couldn't be calculated for LGBMClassifier
multi_class must be in ('ovo', 'ovr')
{'Model': 'LGBMClassifier', 'Accuracy': 0.8001161122874121, 'Balanced Accuracy': 0.7497043571647103, 'ROC AUC': None, 'F1 Score': 0.7972947034484401, 'Time taken': 5.713880300521851}
                               Accuracy  Balanced Accuracy ROC AUC  F1 Score  \
Model                                                                          
XGBClassifier                      0.80               0.75    None      0.80   
LGBMClassifier                     0.80               0.75    None      0.80   
RandomForestClassifier             0.80               0.75    None      0.80   
BaggingClassifier                  0.79               0.74    None      0.79   
ExtraTreesClassifier               0.80               0.73    None      0.79   
SVC                                0.79               0.73    None      0.78   
AdaBoostClassifier                 0.78               0.72    None      0.77   
LogisticR

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model_rf = RandomForestClassifier(n_estimators = 1000, random_state = 42)
model_rf.fit(x_train, y_train)
y_pred_rf = model_rf.predict(x_test)
print(f'The accuracy of the random forest model is {accuracy_score(y_test, y_pred_rf)*100}%.')

In [24]:
!pip install xgboost

Defaulting to user installation because normal site-packages is not writeable


In [25]:
import xgboost as xgb
xgb.__version__

'2.1.0'

In [26]:
model_xgb = xgb.XGBClassifier(
    objective = 'multi:softprob',  # Change this based on your specific problem (e.g., 'multi:softprob' for multi-class classification)
    n_estimators = 1000,
    max_depth = 15,
    learning_rate = 0.1,
    use_label_encoder = False  # Set this to False if you're using XGBoost version >= 1.3.0
)
model_xgb.fit(x_train, y_train)
y_pred_xgb = model_xgb.predict(x_test)
print(f'The accuracy of the xgboost model is {accuracy_score(y_test, y_pred_xgb)*100}%.')

The accuracy of the xgboost model is 81.85233249095008%.


ENSEMBLING THE LIGHTGBM AND XGBOOST MODELS

In [39]:
prob_lgbm = model_lgbm_loaded.predict_proba(x_test)

In [57]:
Y_sma = prob_lgbm[0]
Y_sma[1]

0.9954337435151663

In [40]:
prob_xgb = model_xgb_loaded.predict_proba(x_test)

In [41]:
y_pred_combine = np.argmax((prob_lgbm + prob_xgb)/2, axis = 1)
y_pred_combine

array([1, 2, 0, ..., 2, 2, 2], dtype=int64)

In [42]:
print('For the ensemble model')
print(f'Accuracy = {accuracy_score(y_test, y_pred_combine)*100}%') #best final model. close analysis. combination of lightgbm and xgboost models
print(f'Balanced Accuracy = {balanced_accuracy_score(y_test, y_pred_combine)*100}%')
print(f'Precision = {precision_score(y_test, y_pred_combine, average = mode)*100}%')
print(f'F1 Score = {f1_score(y_test, y_pred_combine, average = mode)*100}%')
confusion_matrix(y_test, y_pred_combine)

For the ensemble model
Accuracy = 82.20066935318626%
Balanced Accuracy = 77.47640068310263%
Precision = 81.8458245702536%
F1 Score = 81.92827842143541%


array([[ 9966,   570,  1054],
       [  878,  2813,  1063],
       [ 1030,   617, 11291]], dtype=int64)

In [43]:
prob_lgbm_train = model_lgbm.predict_proba(x_train)
prob_xgb_train = model_xgb.predict_proba(x_train)

In [44]:
x_new_train = np.concatenate((prob_lgbm_train, prob_xgb_train), axis = 1)

In [45]:
x_comb_train, x_comb_val, y_comb_train, y_comb_val = train_test_split(x_new_train, y_train, test_size = 0.3, random_state = 42)

In [46]:
model_cat = CatBoostClassifier(iterations = 5000, learning_rate = 0.05, depth = 6, eval_metric = 'Accuracy', random_seed = 42, logging_level = 'Verbose', use_best_model = True, early_stopping_rounds = 300)
model_cat.fit(x_comb_train, y_comb_train, eval_set = (x_comb_val, y_comb_val))
y_pred_comb_cat = model_cat.predict(np.concatenate((prob_lgbm, prob_xgb), axis = 1))
accuracy_score(y_test, y_pred_comb_cat)*100

0:	learn: 0.9996738	test: 0.9994877	best: 0.9994877 (0)	total: 105ms	remaining: 8m 44s
1:	learn: 0.9996738	test: 0.9995024	best: 0.9995024 (1)	total: 146ms	remaining: 6m 4s
2:	learn: 0.9997115	test: 0.9995755	best: 0.9995755 (2)	total: 191ms	remaining: 5m 17s
3:	learn: 0.9997365	test: 0.9996048	best: 0.9996048 (3)	total: 222ms	remaining: 4m 37s
4:	learn: 0.9997365	test: 0.9996341	best: 0.9996341 (4)	total: 254ms	remaining: 4m 13s
5:	learn: 0.9997365	test: 0.9996341	best: 0.9996341 (4)	total: 285ms	remaining: 3m 57s
6:	learn: 0.9997115	test: 0.9996341	best: 0.9996341 (4)	total: 318ms	remaining: 3m 46s
7:	learn: 0.9997365	test: 0.9996341	best: 0.9996341 (4)	total: 357ms	remaining: 3m 42s
8:	learn: 0.9997115	test: 0.9996048	best: 0.9996341 (4)	total: 391ms	remaining: 3m 36s
9:	learn: 0.9997115	test: 0.9996048	best: 0.9996341 (4)	total: 423ms	remaining: 3m 30s
10:	learn: 0.9997115	test: 0.9996341	best: 0.9996341 (4)	total: 454ms	remaining: 3m 25s
11:	learn: 0.9997115	test: 0.9996341	best: 

94:	learn: 0.9999498	test: 1.0000000	best: 1.0000000 (70)	total: 3.37s	remaining: 2m 54s
95:	learn: 0.9999498	test: 1.0000000	best: 1.0000000 (70)	total: 3.4s	remaining: 2m 53s
96:	learn: 0.9999498	test: 1.0000000	best: 1.0000000 (70)	total: 3.44s	remaining: 2m 53s
97:	learn: 0.9999498	test: 1.0000000	best: 1.0000000 (70)	total: 3.47s	remaining: 2m 53s
98:	learn: 0.9999498	test: 1.0000000	best: 1.0000000 (70)	total: 3.5s	remaining: 2m 53s
99:	learn: 0.9999498	test: 1.0000000	best: 1.0000000 (70)	total: 3.53s	remaining: 2m 53s
100:	learn: 0.9999498	test: 1.0000000	best: 1.0000000 (70)	total: 3.57s	remaining: 2m 52s
101:	learn: 0.9999498	test: 1.0000000	best: 1.0000000 (70)	total: 3.6s	remaining: 2m 53s
102:	learn: 0.9999498	test: 1.0000000	best: 1.0000000 (70)	total: 3.64s	remaining: 2m 53s
103:	learn: 0.9999498	test: 1.0000000	best: 1.0000000 (70)	total: 3.67s	remaining: 2m 52s
104:	learn: 0.9999498	test: 1.0000000	best: 1.0000000 (70)	total: 3.7s	remaining: 2m 52s
105:	learn: 0.999949

188:	learn: 0.9999498	test: 1.0000000	best: 1.0000000 (70)	total: 6.55s	remaining: 2m 46s
189:	learn: 0.9999498	test: 1.0000000	best: 1.0000000 (70)	total: 6.59s	remaining: 2m 46s
190:	learn: 0.9999498	test: 1.0000000	best: 1.0000000 (70)	total: 6.62s	remaining: 2m 46s
191:	learn: 0.9999498	test: 1.0000000	best: 1.0000000 (70)	total: 6.65s	remaining: 2m 46s
192:	learn: 0.9999498	test: 1.0000000	best: 1.0000000 (70)	total: 6.69s	remaining: 2m 46s
193:	learn: 0.9999498	test: 1.0000000	best: 1.0000000 (70)	total: 6.72s	remaining: 2m 46s
194:	learn: 0.9999498	test: 1.0000000	best: 1.0000000 (70)	total: 6.75s	remaining: 2m 46s
195:	learn: 0.9999498	test: 1.0000000	best: 1.0000000 (70)	total: 6.79s	remaining: 2m 46s
196:	learn: 0.9999498	test: 1.0000000	best: 1.0000000 (70)	total: 6.82s	remaining: 2m 46s
197:	learn: 0.9999498	test: 1.0000000	best: 1.0000000 (70)	total: 6.85s	remaining: 2m 46s
198:	learn: 0.9999498	test: 1.0000000	best: 1.0000000 (70)	total: 6.89s	remaining: 2m 46s
199:	learn

286:	learn: 0.9999561	test: 1.0000000	best: 1.0000000 (70)	total: 9.82s	remaining: 2m 41s
287:	learn: 0.9999561	test: 1.0000000	best: 1.0000000 (70)	total: 9.85s	remaining: 2m 41s
288:	learn: 0.9999624	test: 1.0000000	best: 1.0000000 (70)	total: 9.88s	remaining: 2m 41s
289:	learn: 0.9999624	test: 1.0000000	best: 1.0000000 (70)	total: 9.92s	remaining: 2m 41s
290:	learn: 0.9999624	test: 1.0000000	best: 1.0000000 (70)	total: 9.95s	remaining: 2m 40s
291:	learn: 0.9999624	test: 1.0000000	best: 1.0000000 (70)	total: 9.98s	remaining: 2m 40s
292:	learn: 0.9999624	test: 1.0000000	best: 1.0000000 (70)	total: 10s	remaining: 2m 40s
293:	learn: 0.9999624	test: 1.0000000	best: 1.0000000 (70)	total: 10.1s	remaining: 2m 40s
294:	learn: 0.9999624	test: 1.0000000	best: 1.0000000 (70)	total: 10.1s	remaining: 2m 40s
295:	learn: 0.9999624	test: 1.0000000	best: 1.0000000 (70)	total: 10.1s	remaining: 2m 40s
296:	learn: 0.9999624	test: 1.0000000	best: 1.0000000 (70)	total: 10.2s	remaining: 2m 40s
297:	learn: 

76.23113175329554

In [47]:
x_new_train.shape

(227743, 6)

In [48]:
model_rf = RandomForestClassifier(n_estimators = 100, random_state = 42)
model_rf.fit(x_new_train, y_train)

RandomForestClassifier(random_state=42)

In [49]:
y_pred_rf_combine = model_rf.predict(np.concatenate((prob_lgbm, prob_xgb), axis = 1))
accuracy_score(y_test, y_pred_rf_combine)*100

82.00601051840721

In [35]:
# Save the model to a file
with open('model_lgbm.pkl', 'wb') as f:
    pickle.dump(model_lgbm, f)

In [36]:
# Load the model from the file
mode = 'weighted'
with open('model_lgbm.pkl', 'rb') as f:
    model_lgbm_loaded = pickle.load(f)
y_pred_lgbm = model_lgbm_loaded.predict(x_test)
print(f'Accuracy = {accuracy_score(y_test, y_pred_lgbm)*100}%')
print(f'Balanced Accuracy = {balanced_accuracy_score(y_test, y_pred_lgbm)*100}%')
print(f'Precision = {precision_score(y_test, y_pred_lgbm, average = mode)*100}%')
print(f'F1 Score = {f1_score(y_test, y_pred_lgbm, average = mode)*100}%')

Accuracy = 82.17676388224848%
Balanced Accuracy = 77.4021464350288%
Precision = 81.81557435639448%
F1 Score = 81.8932447713909%


In [37]:
# Save the model to a file
with open('model_xgb.pkl', 'wb') as f:
    pickle.dump(model_xgb, f)

In [38]:
mode = 'weighted'
with open('model_xgb.pkl', 'rb') as f:
    model_xgb_loaded = pickle.load(f)
y_pred_xgb = model_xgb_loaded.predict(x_test)
print(f'Accuracy = {accuracy_score(y_test, y_pred_xgb)*100}%')
print(f'Balanced Accuracy = {balanced_accuracy_score(y_test, y_pred_xgb)*100}%')
print(f'Precision = {precision_score(y_test, y_pred_xgb, average = mode)*100}%')
print(f'F1 Score = {f1_score(y_test, y_pred_xgb, average = mode)*100}%')

Accuracy = 81.85233249095008%
Balanced Accuracy = 77.20575839957341%
Precision = 81.50645842495986%
F1 Score = 81.6005904924521%


In [28]:
!pip install lightgbm

Defaulting to user installation because normal site-packages is not writeable


In [ ]:
del model_lgbm

In [29]:
import lightgbm as lgb
model_lgbm = lgb.LGBMClassifier(
    boosting_type = 'gbdt',
    objective = 'multiclass',  # Change this based on your specific problem (e.g., 'multiclass' for multi-class classification)
    num_leaves = 1000,
    learning_rate = 0.1,
    n_estimators = 1000
)
model_lgbm.fit(x_train, y_train)
y_pred_lgbm = model_lgbm.predict(x_test)
print(f'The accuracy of the LGBM model is {accuracy_score(y_test, y_pred_lgbm)*100}%.')

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.013811 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2342
[LightGBM] [Info] Number of data points in the train set: 227743, number of used features: 39
[LightGBM] [Info] Start training from score -0.911343
[LightGBM] [Info] Start training from score -1.830111
[LightGBM] [Info] Start training from score -0.826403
The accuracy of the LGBM model is 82.17676388224848%.


In [ ]:
del model_bag

In [30]:
from sklearn.ensemble import BaggingClassifier
base_estimator = lgb.LGBMClassifier(
    boosting_type = 'gbdt',
    objective = 'multiclass',  # Change this based on your specific problem (e.g., 'multiclass' for multi-class classification)
    num_leaves = 50,
    learning_rate = 0.05,
    n_estimators = 100
)
model_bag = BaggingClassifier(
    base_estimator=base_estimator,
    n_estimators=50,       # Number of base estimators to train
    max_samples=0.8,       # Fraction of samples to draw from X to train each base estimator
    max_features=1.0,      # Fraction of features to draw from X to train each base estimator
    bootstrap=True,        # Whether samples are drawn with replacement
    bootstrap_features=False,  # Whether features are drawn with replacement
    random_state=42
)
model_bag.fit(x_train, y_train)
y_pred_bag = model_bag.predict(x_test)
print(f'The accuracy of the bagging LGBM model is {accuracy_score(y_test, y_pred_bag)*100}%.')

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.017731 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2342
[LightGBM] [Info] Number of data points in the train set: 227743, number of used features: 39
[LightGBM] [Info] Start training from score -0.918318
[LightGBM] [Info] Start training from score -1.822684
[LightGBM] [Info] Start training from score -0.822758
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.018895 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2341
[LightGBM] [Info] Number of data points in the train set: 227743, number of used features: 39
[LightGBM] [Info] Start training from score -0.915353
[LightGBM] [Info] Start training from score -1.829569
[LightGBM] [Info] Star

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.017009 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2341
[LightGBM] [Info] Number of data points in the train set: 227743, number of used features: 39
[LightGBM] [Info] Start training from score -0.912929
[LightGBM] [Info] Start training from score -1.825098
[LightGBM] [Info] Start training from score -0.826790
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.017430 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2341
[LightGBM] [Info] Number of data points in the train set: 227743, number of used features: 39
[LightGBM] [Info] Start training from score -0.911713
[LightGBM] [Info] Start training from score -1.823567
[LightGBM] [Info] Star

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.014083 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2342
[LightGBM] [Info] Number of data points in the train set: 227743, number of used features: 39
[LightGBM] [Info] Start training from score -0.912656
[LightGBM] [Info] Start training from score -1.827246
[LightGBM] [Info] Start training from score -0.826251
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.015139 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2340
[LightGBM] [Info] Number of data points in the train set: 227743, number of used features: 39
[LightGBM] [Info] Start training from score -0.913832
[LightGBM] [Info] Start training from score -1.819598
[LightGBM] [Info] Star

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.016397 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2342
[LightGBM] [Info] Number of data points in the train set: 227743, number of used features: 39
[LightGBM] [Info] Start training from score -0.905761
[LightGBM] [Info] Start training from score -1.827007
[LightGBM] [Info] Start training from score -0.832704
The accuracy of the bagging LGBM model is 79.94330988320469%.


In [31]:
confusion_matrix(y_test, y_pred_xgb)

array([[ 9925,   598,  1067],
       [  884,  2814,  1056],
       [ 1061,   648, 11229]], dtype=int64)

In [32]:
model_cat.save_model('cat1')

In [33]:
model_loaded = CatBoostClassifier()
model_loaded.load_model('cat1')

In [34]:
model_loaded.predict(x_test)

array([[1],
       [2],
       [0],
       ...,
       [2],
       [2],
       [2]], dtype=int64)